# Elevation Comparison for AGU 2025

## CO_WestCentral_2019

Un-adjusted baseline comparison (just reproject products into same CRS)

In [ ]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [ ]:
import common_functions
from rasterio.enums import Resampling

import coincident
from coincident import pcd_fixtures

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Overview plots 

In [ ]:
site = "CO_WestCentral_2019"

site_meta = pcd_fixtures.read_pcd_site(site)  # dataframes

In [ ]:
# Original search range around ALS dates, and granule names
site_search_results = pcd_fixtures.PCD_SITES[site]
site_search_results

In [ ]:
common_functions.interactive_site_map(site_meta, title=site)

In [ ]:
ax = common_functions.static_site_map(site_meta, title=f"{site} - Footprint Overlap")

In [ ]:
common_functions.plot_timeline(site_meta, search_dates, title=f"{site} - Acquisitions")

## Standard product comparison

As a baseline, we'll compare standard DEM products, IS2, and GEDI


In [ ]:
gf_als, gf_maxar, gf_is2, gf_gedi, gf_overlap = common_functions.load_geodataframes(
    site_meta
)
da_cop = coincident.io.xarray.load_dem_7912("cop30", aoi=gf_overlap)
da_cop

In [ ]:
# 10m
da_lidar = coincident.io.xarray.load_dem_7912("3dep", aoi=gf_overlap)

In [ ]:
da_lidar

In [ ]:
ds_cop_r = (
    da_cop.rio.reproject_match(
        da_lidar,
        resampling=Resampling.bilinear,
    )
    .where(da_lidar.notnull())
    .to_dataset(name="elevation")
)

ds_cop_r

In [ ]:
# Create hillshade variables for plot backgrounds
# This function expects Datasets
ds_lidar = da_lidar.to_dataset(name="elevation")
ds_lidar["hillshade"] = coincident.io.gdal.gdaldem(ds_lidar.elevation, "hillshade")

In [ ]:
## Load altimeters
# Get GEDI
data_gedi = coincident.io.sliderule.subset_gedi02a(
    gf_gedi, gf_overlap, include_worldcover=True
)

In [ ]:
# Get ICSAT-2
data_is2 = coincident.io.sliderule.subset_atl06(
    gf_is2, gf_overlap, include_worldcover=True
)

In [ ]:
test = coincident.plot.utils.sample_dem_at_points(
    da_lidar, data_is2
)  # , diff_col="h_li")
test

In [ ]:
dems = {"3DEP": ds_lidar, "COP": ds_cop_r}
altimeters = {"ICESat-2": (data_is2, "h_li"), "GEDI": (data_gedi, "elevation_lm")}

In [ ]:
# Shashank's recommendation
# Better labels? use ds.name
ax_dict = coincident.plot.compare_dems(
    dems,
    altimeters,
    add_hillshade=True,
    altimetry_basemap="Esri.WorldImagery",
    # altimetry_basemap='hillshade',
    # elevation_clim=(1000, 4000),
    elevation_cmap="plasma",
    # NOTE: best size depends on aspect ratios and number of columns
    figsize=(8.5, 11),
    suptitle=f"Elevation comparisons for {site}",
)

In [ ]:
# More stats on particular comparison
diff = (ds_cop_r.elevation - ds_lidar.elevation).to_series()
ax = coincident.plot.plot_diff_hist(diff, range=(-20, 30))

## STV-generated products


1-meter posting generated with https://github.com/uw-cryo/lidar_tools

In [ ]:
# Requires `stv-user` credentials
import os

print(os.environ.get("AWS_PROFILE"))
!aws s3 ls --human-readable s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/

In [ ]:
!AWS_PROFILE=stv-user gdalinfo /vsis3/uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/CO_WestCentral_2019-DSM_mos.tif

In [ ]:
# Same comparison panel with custom lidar and stereo
# import coincident  # Seems to set global rasterio env
import rasterio
import xarray as xr

href = "s3://uw-cryo-stv/usgs_pcd_products/CO_WestCentral_2019_processing/CO_WestCentral_2019-DSM_mos.tif"


with rasterio.Env(profile_name="stv-user"):
    da_stv_lidar = xr.open_dataarray(
        href,
        engine="rasterio",
        mask_and_scale=False,  # otherwise uint8 -> float32!
        backend_kwargs={"open_kwargs": {"overview_level": 3}},
    )
da_stv_lidar